
<a href="https://colab.research.google.com/github/aviadr1/learn-advanced-python/blob/master/under_development/descriptors.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" 
     title="Open this file in Google Colab" alt="Colab"/>
</a>


# Descriptros

Descriptors are a powerful, general purpose protocol. They are the mechanism behind properties, methods, static methods, class methods, and super(). They are used throughout Python itself to implement the new style classes introduced in version 2.2. Descriptors offer a flexible set of new tools for everyday Python programs.

Simply put, descriptors are objects that know which object owns them 

__get__(self, instance, owner)

* further reading: [link](https://docs.python.org/3.7/howto/descriptor.html)

In [53]:
class Property(object):
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        #ppint('get', self)
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [37]:
class Person:
    def __init__(self, name):
        self._name = name
    def getname(self):
        return self._name
    name = Property(getname)
    friend = Person('friend')
    def __repr__(self):
        return f"{type(self).__name__}({self.name!r})"
    def walk(self):
        print(self.name, 'walking')
    def run(self):
        print(self.name,'running')
    def swim(self):
        print(self.name,'swimming')
        
    def __get__(self, obj, objtype=None):
        print(self, obj, objtype)
        return self
    
    
        
class OlympicRunner(Person):
    def run(self):
        print(self.name,self.name,"running incredibly fast!")
    
def train(person):
    person.walk()
    person.swim()
    person.run()

In [46]:
terry = Person('Terry Gilliam')
terry.name
terry.friend2 = Person('graham chapman')

print('\n', terry.__dict__, '\n' )

print('\n', Person.__dict__, '\n' )

#terry.friend = Person('graham chapman')
terry.friend.walk()


 {'_name': 'Terry Gilliam', 'friend2': Person('graham chapman')} 


 get <__main__.Property object at 0x00FFE110>
{'__module__': '__main__', '__init__': <function Person.__init__ at 0x00EC0DF8>, 'getname': <function Person.getname at 0x00EC0F60>, 'name': <__main__.Property object at 0x00AF8FB0>, 'friend': Person('friend'), '__repr__': <function Person.__repr__ at 0x00EC0C90>, 'walk': <function Person.walk at 0x00EC0C48>, 'run': <function Person.run at 0x00EC0A08>, 'swim': <function Person.swim at 0x00EC0FA8>, '__get__': <function Person.__get__ at 0x00EC0B28>, '__dict__': <attribute '__dict__' of 'Person' objects>, '__weakref__': <attribute '__weakref__' of 'Person' objects>, '__doc__': None} 

get <__main__.Property object at 0x00FFE110>
Person('friend') Person('Terry Gilliam') <class '__main__.Person'>
get <__main__.Property object at 0x00FFE110>
friend walking


In [9]:


type(terry).__dict__['walk'].__get__(terry, type(terry))

<bound method Person.walk of Person('Terry Gilliam')>

In [6]:
type(terry).__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__repr__': <function __main__.Person.__repr__(self)>,
              'walk': <function __main__.Person.walk(self)>,
              'run': <function __main__.Person.run(self)>,
              'swim': <function __main__.Person.swim(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [47]:
def blah(self):
    print(self, 'is blabing')

In [48]:
Person.blah = blah

In [49]:
Person('terry').blah()

Person('terry') is blabing


In [50]:
terry

Person('Terry Gilliam')

In [51]:
Person.blah2 = blah

In [52]:
terry.blah2()

Person('Terry Gilliam') is blabing


In [54]:
def bind(instance, func, as_name=None):
    """
    Bind the function *func* to *instance*, with either provided name *as_name*
    or the existing name of *func*. The provided *func* should accept the 
    instance as the first argument, i.e. "self".
    """
    if as_name is None:
        as_name = func.__name__
    bound_method = func.__get__(instance, instance.__class__)
    setattr(instance, as_name, bound_method)
    return bound_method

def sing(self):
    print(self, 'is singing')
    
bind(terry, sing, 'sing2')
dir(terry.sing2)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [60]:
getattr(OlympicRunner('miew'), 'walk')

<bound method Person.walk of OlympicRunner('miew')>